<a href="https://colab.research.google.com/github/AtomZa/LowNColab/blob/main/DirectedEvo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install git+https://github.com/ElArkk/jax-unirep.git --upgrade
!pip install biopython
!pip install seaborn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/ElArkk/jax-unirep.git to /tmp/pip-req-build-4u1ah1yo
  Running command git clone --filter=blob:none --quiet https://github.com/ElArkk/jax-unirep.git /tmp/pip-req-build-4u1ah1yo
  Resolved https://github.com/ElArkk/jax-unirep.git to commit 7763bf69cc7864f8cf466151e452c52f3adc6476
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install feather-format

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from jax_unirep import get_reps, fit
from jax_unirep.utils import load_params
from Bio import SeqIO
import pandas as pd
import glob
import os
from pathlib import Path
import numpy as np

from sklearn.model_selection import train_test_split, KFold
from sklearn.linear_model import RidgeCV

import warnings
warnings.filterwarnings('ignore') 

import matplotlib.pyplot as plt
import seaborn as sns
import feather
import random


In [ ]:
# read FASTA file:
# input: file name
# output: names and sequences in the file as an array of dim-2 arrays [name, sequence].
def read_fasta(name):
    fasta_seqs = SeqIO.parse(open(name + '.fasta.txt'),'fasta')
    data = []
    for fasta in fasta_seqs:
        data.append([fasta.id, str(fasta.seq).strip()])
    
    return data

# read sequence text file:
# input: file name
# output: names and sequences in the file as an array of dim-2 arrays [name, sequence].
def read_labeled_data(name):
    seqs = np.loadtxt(name + '_seqs.txt', dtype='str')
    
    fitnesses = np.loadtxt(name + '_fitness.txt')
    data = []
    for seq, fitness in zip(seqs, fitnesses):
        data.append([str(seq).strip(), fitness])
    
    return data

# save represented dataframe of features as feather
def save_reps(df, path):
  feather.write_dataframe(df, path + '.feather')
  print(path + '.feather', 'saved!')


# read represented dataframe of features as feather
def read_reps(path):
  return feather.read_dataframe(path + '.feather')


aa_to_int = {
  'M':1,
  'R':2,
  'H':3,
  'K':4,
  'D':5,
  'E':6,
  'S':7,
  'T':8,
  'N':9,
  'Q':10,
  'C':11,
  'U':12,
  'G':13,
  'P':14,
  'A':15,
  'V':16,
  'I':17,
  'F':18,
  'Y':19,
  'W':20,
  'L':21,
  'O':22, #Pyrrolysine
  'X':23, # Unknown
  'Z':23, # Glutamic acid or GLutamine
  'B':23, # Asparagine or aspartic acid
  'J':23, # Leucine or isoleucine
  'start':24,
  'stop':25,
}


def get_int_to_aa():
  return {value:key for key, value in aa_to_int.items()}


def _one_hot(x, k, dtype=np.float32):
  return np.array(x[:, None] == np.arange(k))


def aa_seq_to_int(s):
  """Return the int sequence as a list for a given string of amino acids."""
  # Make sure only valid aa's are passed
  if not set(s).issubset(set(aa_to_int.keys())):
    raise ValueError(
      f"Unsupported character(s) in sequence found:"
      f" {set(s).difference(set(aa_to_int.keys()))}"
    )

  return [aa_to_int[a] for a in s]


def aa_seq_to_onehot(seq):
  return 1*np.equal(np.array(aa_seq_to_int(seq))[:,None], np.arange(21)).flatten()
  

def multi_onehot(seqs):
  return np.stack([aa_seq_to_onehot(s) for s in seqs.tolist()])


def confusion_matrix_loss(Y_test,Y_preds_test):

  N = len(Y_test)
  Y_rank_matrix = np.zeros((N,N))
  Y_preds_rank_matrix = np.zeros((N,N))
  for i in range(N):
    for j in range(N):

      if Y_test[i] > Y_test[j]:
        Y_rank_matrix[i,j] = 1
      elif Y_test[i] <= Y_test[j]:
        Y_rank_matrix[i,j] = 0
      if Y_preds_test[i] > Y_preds_test[j]:
        Y_preds_rank_matrix[i,j] = 1
      elif Y_preds_test[i] <= Y_preds_test[j]:
        Y_preds_rank_matrix[i,j] = 0
  confusion_matrix = ~(Y_preds_rank_matrix == Y_rank_matrix)

  loss = np.sum(confusion_matrix)/confusion_matrix.size

  return loss

#Load the functions and utilities needed to simulate DE

In [ ]:
def get_top_model(df, alpha, train_batch_size):

  rand_state_num = 42

  np.random.seed(rand_state_num)

  rndperm = np.random.permutation(df.shape[0])

  X_train = df.loc[rndperm[:train_batch_size], df.columns[2:]]
  Y_train = df.loc[rndperm[:train_batch_size], "fitness"]

  kfold = KFold(n_splits=10, random_state=rand_state_num, shuffle=True)

  return RidgeCV(alphas=alpha, cv=kfold).fit(X_train, Y_train)

def init_by_rep(df, alpha_val, N, param_file):

  if param_file == None:
    params = load_params(None, 1900)
    DE_model = get_top_model(df, [alpha_val], N) # choose unirep representation, alpha=1e-3, and 96 training mutants


  else: # if we want to use an evotuned representation:
    params = load_params(param_file, 1900) # param_file
    DE_model = get_top_model(df, [alpha_val], N) # choose eunirep representation, alpha=1e-3, and 96 training mutants

  return params, DE_model


def mutate_sequence(seq,m,prev_mut_loc): # produce a mutant sequence (integer representation), given an initial sequence and the number of mutations to introduce ("m")

  for i in range(m): #iterate through number of mutations to add
    rand_loc = random.randint(prev_mut_loc-8,prev_mut_loc+8) # find random position to mutate
    while (rand_loc <=0) or (rand_loc >= len(seq)):
      rand_loc = random.randint(prev_mut_loc-8,prev_mut_loc+8)

    rand_aa = random.randint(1,21) # find random amino acid to mutate to
    seq = list(seq)
    seq[rand_loc] = get_int_to_aa()[rand_aa] # update sequence to have new amino acid at randomely chosen position
    seq = ''.join(seq)


  return seq,rand_loc # output the randomely mutated sequence

def directed_evolution(s_wt,num_iterations,T,Model, params): # input = (wild-type sequence, number of mutation iterations, "temperature")		


  s_traj = [] # initialize an array to keep records of the protein sequences for this trajectory
  y_traj = [] # initialize an array to keep records of the fitness scores for this trajectory


  mut_loc_seed = random.randint(0,len(s_wt)) # randomely choose the location of the first mutation in the trajectory
  s,new_mut_loc = mutate_sequence(s_wt, (np.random.poisson(2) + 1),mut_loc_seed) # initial mutant sequence for this trajectory, with m = Poisson(2)+1 mutations

  x,_,_ = get_reps([s],params=params)# eUniRep representation of the initial mutant sequence for this trajectory

  y = Model.predict(x) # predicted fitness score for the initial mutant sequence for this trajectory


  # iterate through the trial mutation steps for the directed evolution trajectory
  for i in range(num_iterations):
    mu = np.random.uniform(1,2.5) # "mu" parameter for poisson function: used to control how many mutations to introduce
    m = np.random.poisson(mu-1) + 1 # how many random mutations to apply to current sequence

    s_new,new_mut_loc = mutate_sequence(s, m, new_mut_loc) # new trial sequence, produced from "m" random mutations

    x_new,_,_ = get_reps([s_new],params=None)

    y_new = Model.predict(x_new) # new fitness value for trial sequence

    p = min(1,np.exp((y_new-y)/T)) # probability function for trial sequence
    rand_var = random.random()

    if rand_var < p: # metropolis-Hastings update selection criterion
      print(str(new_mut_loc+1)+" "+s[new_mut_loc]+"->"+s_new[new_mut_loc])
      s, y = s_new, y_new # if criteria is met, update sequence and corresponding fitness

    s_traj.append(s) # update the sequence trajectory records for this iteration of mutagenesis
    y_traj.append(y) # update the fitness trajectory records for this iteration of mutagenesis


  return s_traj, y_traj # output = (sequence record for trajectory, fitness score recorf for trajectory)


def run_DE_trajectories(s_wt, Model, T, num_iterations, num_trajectories, DE_record_folder,params,save=False):

  s_records = [] # initialize list of sequence records
  y_records = [] # initialize list of fitness score records

  for i in range(num_trajectories): #iterate through however many mutation trajectories we want to sample
    s_traj, y_traj = directed_evolution(s_wt,num_iterations,T,Model,params) # call the directed evolution function, outputting the trajectory sequence and fitness score records

    s_records.append(s_traj) # update the sequence trajectory records for this full mutagenesis trajectory
    y_records.append(y_traj) # update the fitness trajectory records for this full mutagenesis trajectory

    if save==True:
      # iteration_path = directory_path+"DE_records/"
      np.savetxt(DE_record_folder + "/trajectory"+str(i)+"_seqs.txt", np.array(s_traj),fmt="%s")   # save sequence records for trajectory-i
      np.savetxt(DE_record_folder + "/trajectory"+str(i)+"_fitness.txt", np.array(y_traj))   # save fitness records for trajecroty-i
    print("finished trajectory #",i)

  s_records = np.array(s_records)
  y_records = np.array(y_records)

  plt.clf()
  fig = plt.figure(figsize=(10,6))
  plt.plot(np.transpose(y_records[:,:,0])) # plot the changes in fitness for all sampled trajectories
  plt.ylabel('Predicted Fitness')
  plt.xlabel('Mutation Trial Steps')
  plt.show() # show the plot :)

  return s_records, y_records

# TEM-1 beta-lactamase example:
## Mount google drive, Load datasets and parameters:


In [ ]:
directory_path = '/content/sample_data/testrun/'


seqs_df = pd.DataFrame(read_labeled_data(directory_path+'TEM1'), columns = ['sequence', 'fitness'])
df = read_reps(directory_path + 'TEM1_epoch1')

param_file = directory_path + 'TEM1_epoch1_weights'

In [ ]:
alpha = 1e-2

BATCH_SIZE = 85

rand_state_num = 42

In [ ]:
np.random.seed(rand_state_num)
rndperm = np.random.permutation(df.shape[0])


TRAIN_BATCH_SIZE = int(BATCH_SIZE*0.8)
HOLDOUT_BATCH_SIZE = int(BATCH_SIZE*0.2)

training_df = df.iloc[rndperm[:TRAIN_BATCH_SIZE],:]
                          
testing_df = df.iloc[rndperm[TRAIN_BATCH_SIZE:TRAIN_BATCH_SIZE+HOLDOUT_BATCH_SIZE],:]

params, DE_model = init_by_rep(df, alpha, TRAIN_BATCH_SIZE, param_file)


#Make a plot to check how well the top model will perform


Train set ranking error:  0.10532006920415225
Test set ranking error:  0.31141868512110726


In [ ]:
X_train = training_df.loc[:, training_df.columns[2:]]
Y_train = training_df.loc[:, "fitness"]
X_test = testing_df.loc[:, training_df.columns[2:]]
Y_test = testing_df.loc[:, "fitness"]


Y_train_preds = DE_model.predict(X_train)
Y_test_preds = DE_model.predict(X_test)

print("Train set ranking error: ", confusion_matrix_loss(np.array(Y_train),Y_train_preds))
print("Test set ranking error: ", confusion_matrix_loss(np.array(Y_test),Y_test_preds))


fig = plt.figure(figsize=(8,12))

ax1 = fig.add_subplot(2,1,1)
ax1.plot(np.arange(len(Y_train)), np.array(Y_train)[np.argsort(Y_train)],color='mediumturquoise',linewidth=1.75, alpha=0.8)
ax1.plot(np.arange(len(Y_train)), np.array(Y_train_preds)[np.argsort(Y_train)],color='orchid',linewidth=1.75, alpha=0.8)


ax1.scatter(np.arange(len(Y_train)), np.array(Y_train)[np.argsort(Y_train)],color='mediumturquoise',s=50, alpha=0.5)
ax1.scatter(np.arange(len(Y_train)), np.array(Y_train_preds)[np.argsort(Y_train)],color='orchid',s=50, alpha=0.5)


ax2 = fig.add_subplot(2,1,2)
ax2.plot(np.arange(len(Y_test)), np.array(Y_test)[np.argsort(Y_test)],color='mediumturquoise',linewidth=2.75, alpha=0.8)
ax2.plot(np.arange(len(Y_test)), np.array(Y_test_preds)[np.argsort(Y_test)],color='orchid',linewidth=2.75, alpha=0.8)

ax2.scatter(np.arange(len(Y_test)), np.array(Y_test)[np.argsort(Y_test)],color='mediumturquoise',s=50, alpha=0.5,marker="s")
ax2.scatter(np.arange(len(Y_test)), np.array(Y_test_preds)[np.argsort(Y_test)],color='orchid',s=50, alpha=0.5,marker="s")




ax1.set_ylabel("Training Set \n Fitness Scores",size=16)
ax2.set_ylabel("Validation Set \n Fitness Scores",size=16)


ax1.set_yticklabels(np.round(ax1.get_yticks(),1),fontsize=16)
ax2.set_yticklabels(np.round(ax2.get_yticks(),1),fontsize=16)



ax1.set_xticklabels(ax1.get_xticks().astype(int),fontsize=17)
ax2.set_xticklabels(ax2.get_xticks().astype(int),fontsize=17)


fig.tight_layout(w_pad=2.5, h_pad=2.5)

ax1.legend(['Experimental Values','Predicted Values'],fontsize=14)
ax2.legend(['Experimental Values','Predicted Values'],fontsize=14)


plt.show()

In [ ]:
# define the wild-type sequence
s_wt = "MSIQHFRVALIPFFAAFCLPVFAHPETLVKVKDAEDQLGARVGYIELDLNSGKILESFRPEERFPMMSTFKVLLCGAVLSRVDAGQEQLGRRIHYSQNDLVEYSPVTEKHLTDGMTVRELCSAAITMSDNTAANLLLTTIGGPKELTAFLHNMGDHVTRLDRWEPELNEAIPNDERDTTMPAAMATTLRKLLTGELLTLASRQQLIDWMEADKVAGPLLRSALPAGWFIADKSGAGERGSRGIIAALGPDGKPSRIVVIYTTGSQATMDERNRQIAEIGASLIKHW"


DE_record_folder = directory_path + "TEM1_DE_tutorial_records" # assign project name for the simulation

T = 0.01 # "temperature" parameter: determines sensitivity of Metropolis-Hastings acceptance criteria
num_iterations = 25 # how many subsequent mutation trials per simulated evolution trajectory
num_trajectories = 5 # how many separate evolution trajectories to run

In [ ]:
df

,sequence,fitness,feat1,feat2,feat3,feat4,feat5,feat6,feat7,feat8,...,feat1891,feat1892,feat1893,feat1894,feat1895,feat1896,feat1897,feat1898,feat1899,feat1900
0,ASIQHFRVALIPFFAAFCLPVFAHPETLVKVKDAEDQLGARVGYIE...,0.002018,0.083429,0.017790,0.139186,-0.091438,-0.293297,0.109081,-0.097349,0.012116,...,0.484805,0.096905,-0.027604,0.039013,-0.093324,-0.004980,0.028618,-0.045363,-0.154246,-0.023591
1,CSIQHFRVALIPFFAAFCLPVFAHPETLVKVKDAEDQLGARVGYIE...,0.003023,0.070089,0.003891,0.125863,-0.083742,-0.315909,0.146599,-0.120089,0.016004,...,0.443611,0.110881,-0.065987,0.046150,-0.101544,-0.008828,0.039054,-0.033971,-0.164325,-0.024140
2,DSIQHFRVALIPFFAAFCLPVFAHPETLVKVKDAEDQLGARVGYIE...,0.002220,0.070518,0.004872,0.126074,-0.083650,-0.309558,0.151648,-0.112371,0.013564,...,0.446197,0.109314,-0.072900,0.047054,-0.100211,-0.008950,0.048964,-0.034887,-0.165173,-0.023962
3,ESIQHFRVALIPFFAAFCLPVFAHPETLVKVKDAEDQLGARVGYIE...,0.002378,0.070221,0.005157,0.127237,-0.083484,-0.307628,0.150690,-0.110858,0.012418,...,0.446856,0.110949,-0.071082,0.048089,-0.100278,-0.009531,0.049739,-0.038007,-0.165111,-0.023787
4,FSIQHFRVALIPFFAAFCLPVFAHPETLVKVKDAEDQLGARVGYIE...,0.005702,0.069489,0.004044,0.127809,-0.083865,-0.315446,0.148395,-0.112256,0.012260,...,0.444949,0.109711,-0.069350,0.047181,-0.101089,-0.012605,0.042397,-0.033402,-0.163027,-0.023759
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5464,MSIQHFRVALIPFFAAFCLPVFAHPETLVKVKDAEDQLGARVGYIE...,0.012939,0.084384,0.015481,0.144268,-0.094189,-0.290472,0.106152,-0.095968,-0.004053,...,0.489336,0.098581,-0.019478,0.036913,-0.095845,-0.002898,0.019763,-0.048637,-0.153243,-0.024366
5465,MSIQHFRVALIPFFAAFCLPVFAHPETLVKVKDAEDQLGARVGYIE...,0.004548,0.084374,0.015624,0.144101,-0.094283,-0.290185,0.106130,-0.095793,-0.003694,...,0.489701,0.098568,-0.018396,0.035992,-0.096388,-0.002691,0.019755,-0.048657,-0.153290,-0.024359
5466,MSIQHFRVALIPFFAAFCLPVFAHPETLVKVKDAEDQLGARVGYIE...,0.003689,0.084364,0.015568,0.144089,-0.094277,-0.290498,0.106126,-0.095706,-0.003535,...,0.490040,0.098605,-0.018411,0.035760,-0.096400,-0.002839,0.019759,-0.048658,-0.153215,-0.024358
5467,MSIQHFRVALIPFFAAFCLPVFAHPETLVKVKDAEDQLGARVGYIE...,0.005659,0.084425,0.015466,0.144310,-0.094283,-0.289868,0.106185,-0.095396,-0.003073,...,0.490365,0.098627,-0.018594,0.035648,-0.096539,-0.002803,0.019761,-0.048668,-0.153255,-0.024366


# Now we can run the simulation


66 M->C
62 E->K
63 R->A
59 R->L
70 F->N
62 K->F
63 A->K
71 K->Y
67 A->C
finished trajectory # 0
203 S->R
210 E->L
212 D->M
219 L->Q
218 L->M
217 Y->P
223 L->R
232 K->S
238 R->K
230 A->M
225 A->L
223 R->W
226 T->I
234 G->Y
225 L->L
221 Y->L
finished trajectory # 1
157 V->A
157 A->M
155 D->A
153 M->M
158 T->K
147 T->F
133 A->W
137 L->V
120 L->L
finished trajectory # 2
257 V->H
265 Q->H
257 H->W
249 P->V
245 A->M
249 V->M
244 I->W
238 R->P
255 R->S
260 Y->L
finished trajectory # 3
31 V->R
37 Q->C
41 D->T
48 D->U
54 I->F
52 G->Q
50 N->N
51 S->G
57 D->C
60 P->H
63 R->K
57 Q->A
55 L->L
47 L->A
48 U->T
finished trajectory # 4


<Figure size 640x480 with 0 Axes>

In [ ]:
s_records, y_records = run_DE_trajectories(s_wt, DE_model, T, num_iterations, num_trajectories, DE_record_folder, None ,save=True)


# We can also re-load the data from our previous simulation if needed for subsequent analyses

##define the previous test's info:


In [ ]:
DE_record_folder = directory_path + "TEM1_DE_tutorial_records" # assign project name for the simulation
num_trajectories = 5

## plot the mutation trajectories from the previous test


<Figure size 640x480 with 0 Axes>

In [ ]:
pal = sns.color_palette("husl", num_trajectories)
plt.clf()
fig = plt.figure(figsize=(9,6))
label_list = []
color_idx = np.linspace(0, 1, num_trajectories)
for i,i_color in zip(list(np.arange(num_trajectories)),pal):
  traj_name = DE_record_folder+"/trajectory"+str(i)
  seqs = np.loadtxt(traj_name + "_seqs.txt",dtype=str)
  fitness = np.loadtxt(traj_name + "_fitness.txt")
  plt.plot(fitness,color=i_color,linewidth=2.5,alpha=0.8)
  label_list.append("Trajectory "+str(i+1))
plt.ylabel("Predicted Fitness")
plt.xlabel('Mutation Trial Steps')
plt.legend(label_list)


plt.show()

## We can also print the sequences generated in previous simulations:


In [ ]:
for i in range(num_trajectories):
  print("Trajectory " + str(i+1) +" sequences:")
  seqs = np.loadtxt(DE_record_folder+"/trajectory"+str(1) + "_seqs.txt",dtype=str)
  print(seqs)
  print(" ")

Trajectory 1 sequences:
['MSIQHFRVALIPFFAAFCLPVFAHPETLVKVKDAEDQLGARVGYIELDLNSGKILESFRPEERFPMMSTFKVLLCGAVLSRVDAGQEQLGRRIHYSQNDLVEYSPVTEKHLTDGMTVRELCSAAITMSDNTAANLLLTTIGGPKELTAFLHNMGDHVTRLDRWEPELNEAIPNDERDTTMPAAMATTLRKLLTGELLTLASRSQLIDFWEADKVAGPLLRSALPAGWFIADKSGAGERGSRGIIAALGPDGKPSRIVVIYTTGSQATMDERNRQIAEIGASLIKHW'
 'MSIQHFRVALIPFFAAFCLPVFAHPETLVKVKDAEDQLGARVGYIELDLNSGKILESFRPEERFPMMSTFKVLLCGAVLSRVDAGQEQLGRRIHYSQNDLVEYSPVTEKHLTDGMTVRELCSAAITMSDNTAANLLLTTIGGPKELTAFLHNMGDHVTRLDRWEPELNEAIPNDERDTTMPAAMATTLRKLLTGELLTLASRRQLIDFWEADKVAGPLLRSALPAGWFIADKSGAGERGSRGIIAALGPDGKPSRIVVIYTTGSQATMDERNRQIAEIGASLIKHW'
 'MSIQHFRVALIPFFAAFCLPVFAHPETLVKVKDAEDQLGARVGYIELDLNSGKILESFRPEERFPMMSTFKVLLCGAVLSRVDAGQEQLGRRIHYSQNDLVEYSPVTEKHLTDGMTVRELCSAAITMSDNTAANLLLTTIGGPKELTAFLHNMGDHVTRLDRWEPELNEAIPNDERDTTMPAAMATTLRKLLTGELLTLASRRQLIDFWLADKVAGPLLRSALPAGWFIADKSGAGERGSRGIIAALGPDGKPSRIVVIYTTGSQATMDERNRQIAEIGASLIKHW'
 'MSIQHFRVALIPFFAAFCLPVFAHPETLVKVKDAEDQLGARVGYIELDLNSGKILESFRPEERFPMMSTFKVLLCGAVLSRVDAGQEQLGRRIHYSQNDLVEYS

In [ ]:
params

[Array([[-4.32526559e-01,  7.53880665e-02, -2.11843640e-01,
          6.06281981e-02,  2.77478129e-01, -8.81439075e-02,
          1.04718730e-01,  1.55644700e-01,  5.23038745e-01,
         -3.18140000e-01],
        [ 4.48811613e-02, -3.00060481e-01, -1.45759070e-02,
          1.20491721e-01, -3.09778657e-02,  5.83414081e-03,
         -1.11657895e-01, -5.27199030e-01, -4.85000171e-04,
         -1.91193491e-01],
        [-3.17000806e-01,  1.15881167e-01, -8.25776905e-03,
          5.00198007e-01, -5.91420457e-02,  1.04788348e-01,
         -2.08030924e-01, -2.43377000e-01,  1.65767953e-01,
         -2.02437952e-01],
        [ 5.75120747e-02, -2.68256575e-01,  1.71969414e-01,
          2.34683350e-01,  2.23891228e-01,  8.98892432e-02,
          2.20527779e-02, -3.40744466e-01, -1.12043381e-01,
          1.92534237e-03],
        [ 2.48875588e-01,  4.87032175e-01,  3.21024805e-01,
          9.91747975e-02,  3.95841241e-01,  2.97862347e-02,
          8.90410990e-02, -5.97915724e-02,  3.610234